In [4]:
import os
import json
import random
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from tqdm import tqdm
from typing import Union
from rasterio.features import geometry_mask
from rasterio.windows import Window
import cv2
from rasterio.warp import calculate_default_transform, reproject, Resampling, transform_geom

In [5]:
image_folder = './mydata/imagess'

In [6]:
tile_images = []

In [7]:
for filename in os.listdir(image_folder):
    if filename.endswith('.tif'):
        tile_images.append(os.path.join(image_folder, filename))

In [8]:
print(tile_images)

['./mydata/imagess\\._data_00001.tif', './mydata/imagess\\._data_00002.tif', './mydata/imagess\\._data_00003.tif', './mydata/imagess\\._data_00004.tif', './mydata/imagess\\._data_00005.tif', './mydata/imagess\\._data_00006.tif', './mydata/imagess\\._data_00007.tif', './mydata/imagess\\._data_00008.tif', './mydata/imagess\\._data_00009.tif', './mydata/imagess\\._data_00010.tif', './mydata/imagess\\._data_00011.tif', './mydata/imagess\\._data_00012.tif', './mydata/imagess\\._data_00013.tif', './mydata/imagess\\._data_00014.tif', './mydata/imagess\\._data_00015.tif', './mydata/imagess\\._data_00016.tif', './mydata/imagess\\._data_00017.tif', './mydata/imagess\\._data_00018.tif', './mydata/imagess\\._data_00019.tif', './mydata/imagess\\._data_00020.tif', './mydata/imagess\\._data_00021.tif', './mydata/imagess\\._data_00022.tif', './mydata/imagess\\._data_00023.tif', './mydata/imagess\\._data_00024.tif', './mydata/imagess\\._data_00025.tif', './mydata/imagess\\._data_00026.tif', './mydata/i

In [9]:
with rasterio.open(tile_images[0]) as src:
    profile = src.profile
    crs = src.crs
    transform = src.transform
    height, width = src.shape

In [10]:
output_file = './dataset/output5.tif'

In [11]:
# Calculate the dimensions of the output raster
total_tiles = len(tile_images)
cols = int(total_tiles ** 0.5)
rows = (total_tiles + cols - 1) // cols

In [12]:
print(total_tiles)

12101


In [13]:
# Update the size of the output raster
profile.update(driver='GTiff',
               height=rows * height,
               width=cols * width,
               transform=transform,
               crs=crs,
               count=3)  # Assuming 3 channels for RGB

In [14]:
with rasterio.open(output_file, 'w', **profile) as dst:
    # Step 4: Write each tile image to the appropriate location in the new TIF image
    for i, tile_image in enumerate(tile_images):
        with rasterio.open(tile_image) as src:
            tile_data = src.read()
            window = Window(i % cols * width, i // cols * height, width, height)
            dst.write(tile_data, indexes=[1, 2, 3], window=window)